In [ ]:
!pip install mljar-supervised
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import LabelEncoder
from supervised import AutoML
import torch
from sklearn.preprocessing import OneHotEncoder
from category_encoders.target_encoder import TargetEncoder

N_THREADS = 20
N_FOLDS = 5
RANDOM_STATE = 42
TEST_SIZE = 0.2
TIMEOUT = 3*3600
TARGET_NAME = 'ECLO'

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

train_df=pd.read_csv("/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/train.csv")
test_df=pd.read_csv("/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/test.csv")
cctv=pd.read_csv("/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/CCTV 밀도.csv")

In [ ]:
cctv=pd.read_csv("/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/CCTV 밀도.csv")[['구', '밀도','정규화밀도']]

cctv.rename(columns={'구': '군'}, inplace=True)

cctv = cctv.groupby(['군']).agg({'밀도':'first','정규화밀도':'first'}).reset_index()

In [ ]:
import holidays
def make_holi(x):
    kr_holi = holidays.KR()

    if x in kr_holi:
        return 1
    else:
        return 0

train_df["holiday"] = train_df["사고일시"].map(lambda x : make_holi(x))
test_df["holiday"] = test_df["사고일시"].map(lambda x : make_holi(x))


In [ ]:
#파생 변수 생성 1 : 날짜, 시간정보 생성
time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train_df['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test_df[['연', '월', '일', '시간']] = test_df['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])


In [ ]:
#파생 변수 생성 2 : 군, 구로 나누기

train_df['군'] = ''
train_df['구'] = ''

test_df['군'] = ''
test_df['구'] = ''

# 주소 분리 및 군과 구 열에 저장
for i in range(0, len(train_df)):
    address = train_df['시군구'][i]
    elements = address.split(" ")
    if len(elements) == 3:
        city = elements[0]
        county = elements[1]
        district = elements[2]
        train_df.at[i, '군'] = county
        train_df.at[i, '구'] = district
    else:
        print(f"잘못된 주소 형식입니다: {address}")

# 결과 출력
print(train_df)

                   ID   요일 기상상태              시군구             도로형태 노면상태  사고유형  \
0      ACCIDENT_00000  화요일   맑음     대구광역시 중구 대신동         단일로 - 기타   건조  차대사람   
1      ACCIDENT_00001  화요일   흐림    대구광역시 달서구 감삼동         단일로 - 기타   건조  차대사람   
2      ACCIDENT_00002  화요일   맑음    대구광역시 수성구 두산동         단일로 - 기타   건조  차대사람   
3      ACCIDENT_00003  화요일   맑음     대구광역시 북구 복현동         단일로 - 기타   건조   차대차   
4      ACCIDENT_00004  화요일   맑음     대구광역시 동구 신암동         단일로 - 기타   건조   차대차   
...               ...  ...  ...              ...              ...  ...   ...   
39604  ACCIDENT_39604  금요일   맑음  대구광역시 수성구 수성동3가       교차로 - 교차로안   건조   차대차   
39605  ACCIDENT_39605  금요일   맑음    대구광역시 달서구 상인동         단일로 - 기타   건조   차대차   
39606  ACCIDENT_39606  금요일   맑음    대구광역시 달서구 월성동       교차로 - 교차로안   건조   차대차   
39607  ACCIDENT_39607  금요일   맑음     대구광역시 달서구 장동          기타 - 기타   건조   차대차   
39608  ACCIDENT_39608  금요일   맑음     대구광역시 서구 비산동  단일로 - 지하차도(도로)내   건조   차대차   

      사고유형 - 세부분류       법규위반 가해운전자 차종  

In [ ]:
for i in range(0, len(test_df)):
    address = test_df['시군구'][i]
    elements = address.split(" ")
    if len(elements) == 3:
        city = elements[0]
        county = elements[1]
        district = elements[2]
        test_df.at[i, '군'] = county
        test_df.at[i, '구'] = district
    else:
        print(f"잘못된 주소 형식입니다: {address}")

# 결과 출력
print(test_df)

                   ID   요일 기상상태              시군구         도로형태 노면상태  사고유형  \
0      ACCIDENT_39609  토요일   맑음     대구광역시 수성구 상동   교차로 - 교차로안   건조  차대사람   
1      ACCIDENT_39610  토요일   맑음    대구광역시 수성구 지산동     단일로 - 기타   건조  차대사람   
2      ACCIDENT_39611  토요일   맑음  대구광역시 수성구 수성동2가   교차로 - 교차로안   건조   차대차   
3      ACCIDENT_39612  토요일   맑음    대구광역시 수성구 신매동     단일로 - 기타   건조   차대차   
4      ACCIDENT_39613  토요일   맑음    대구광역시 달서구 감삼동   교차로 - 교차로안   건조   차대차   
...               ...  ...  ...              ...          ...  ...   ...   
10958  ACCIDENT_50567  토요일   맑음     대구광역시 남구 대명동     단일로 - 터널   건조   차대차   
10959  ACCIDENT_50568  토요일   맑음    대구광역시 수성구 시지동     단일로 - 기타   건조   차대차   
10960  ACCIDENT_50569  토요일   맑음    대구광역시 수성구 연호동     단일로 - 기타   건조   차대차   
10961  ACCIDENT_50570  토요일   맑음    대구광역시 수성구 범물동  교차로 - 교차로부근   건조   차대차   
10962  ACCIDENT_50571  토요일   맑음     대구광역시 동구 효목동  교차로 - 교차로부근   건조   차대차   

       holiday     연   월   일  시간    군      구  
0            1  2022   1   1   1  수성구   

In [ ]:
test_df = pd.merge(test_df, cctv, how='left', on=['군'])
train_df = pd.merge(train_df, cctv, how='left', on=['군'])

In [ ]:
#파생 변수 추출 3 : 도로 형태 정보 추출
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_df['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_df['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

In [ ]:
# 파생변수4: 사고 발생 시간대 생성
def classify_time_of_day(hour):
    if 0 <= hour < 6:
        return '새벽'
    elif 6 <= hour < 12:
        return '오전'
    elif 12 <= hour < 18:
        return '오후'
    else:
        return '저녁'

# '사고 발생 시간대' 파생 변수 추가
train_df['사고 발생 시간대'] = train_df['시간'].apply(lambda x: classify_time_of_day(x))
test_df['사고 발생 시간대'] = test_df['시간'].apply(lambda x: classify_time_of_day(x))

In [ ]:
# 파생변수 5: ECLO가 높은 주말에 가중치 부여
train_df['요일 가중치'] = train_df['요일'].apply(lambda x: 1.2 if x in ['토요일', '일요일'] else 1.0)
test_df['요일 가중치'] = test_df['요일'].apply(lambda x: 1.2 if x in ['토요일', '일요일'] else 1.0)

In [ ]:
# 파생변수 6: 교차로, 구조물이 있을 때 가중치 부여
train_df['Crossroad'] = train_df['도로형태1'].apply(lambda x: 1 if '교차로' in x else 0)
# 기타 데이터를 제외한 경우에 가중치를 두 배로 주는 가중치 변수 생성
train_df['도로형태 가중치'] = train_df.apply(lambda row: 1.2 if row['Crossroad'] == 1 and '기타' not in row['도로형태2'] else 1, axis=1)
train_df = train_df.drop(columns=['Crossroad'])

test_df['Crossroad'] = test_df['도로형태1'].apply(lambda x: 1 if '교차로' in x else 0)
# 기타 데이터를 제외한 경우에 가중치를 두 배로 주는 가중치 변수 생성
test_df['도로형태 가중치'] = test_df.apply(lambda row: 1.2 if row['Crossroad'] == 1 and '기타' not in row['도로형태2'] else 1, axis=1)
test_df = test_df.drop(columns=['Crossroad'])

In [ ]:
light_df=pd.read_csv("/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/대구 보안등 정보.csv",encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

light_df[['도시', '군', '구', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])

light_df = light_df.groupby(['도시', '군', '구']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)

<ipython-input-21-5e46896d2f68>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
child_area_df=pd.read_csv("/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/대구 어린이 보호 구역 정보.csv",encoding='cp949').drop_duplicates()[['소재지지번주소']]
child_area_df['cnt'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_area_df[['도시', '군', '구', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])

child_area_df = child_area_df.groupby(['도시', '군', '구']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

In [ ]:
parking_df=pd.read_csv("/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/대구 주차장 정보.csv",encoding='cp949')[['소재지지번주소', '급지구분']]

parking_df = pd.get_dummies(parking_df, columns=['급지구분'])

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

parking_df[['도시', '군', '구', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

parking_df = parking_df.groupby(['도시', '군', '구']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df = pd.merge(train_df, light_df, how='left', on=['군', '구'])
train_df = pd.merge(train_df, child_area_df, how='left', on=['군', '구'])
train_df = pd.merge(train_df, parking_df, how='left', on=['군', '구'])

test_df = pd.merge(test_df, light_df, how='left', on=['군', '구'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['군', '구'])
test_df = pd.merge(test_df, parking_df, how='left', on=['군', '구'])

In [ ]:
# train_df와 test_df의 기상상태 및 노면상태 열 선택
train_categorical_data = train_df[['군']]
test_categorical_data = test_df[['군']]

# OneHotEncoder 인스턴스 생성 및 fit_transform 수행
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_categorical_data)
test_encoded = encoder.transform(test_categorical_data)

# OneHotEncoder가 사용한 카테고리 목록을 가져와서 카테고리 이름을 열 이름으로 변환
feature_names = encoder.get_feature_names_out(['군'])

# 밀집 행렬로 변환 (선택 사항)
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

# 데이터프레임으로 변환 (선택 사항)
import pandas as pd

train_encoded_df = pd.DataFrame(train_encoded_dense, columns=feature_names, index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=feature_names, index=test_df.index)

# 기존 열 제거
train_df = train_df.drop(['군', ], axis=1)
test_df = test_df.drop(['군'], axis=1)

# 인코딩된 열 추가
train_df = pd.concat([train_df, train_encoded_df], axis=1)
test_df = pd.concat([test_df, test_encoded_df], axis=1)


In [ ]:
accident_counts = train_df['시군구'].value_counts().reset_index()
accident_counts.columns = ['시군구', '사고횟수']

# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
train_df['사고발생횟수'] = train_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
test_df['사고발생횟수'] = test_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])

In [ ]:
def group_season(df):
    df.loc[(df['월'] == 3) | (df['월'] == 4) | (df['월'] == 5), 'season'] = '봄'
    df.loc[(df['월'] == 6) | (df['월'] == 7) | (df['월'] == 8), 'season'] = '여름'
    df.loc[(df['월'] == 9) | (df['월'] == 10) | (df['월'] == 11), 'season'] = '가을'
    df.loc[(df['월'] == 12) | (df['월'] == 1) | (df['월'] == 2), 'season'] = '겨울'
    return df['season']

train_df['season'] = group_season(train_df)
test_df['season'] = group_season(test_df)

In [ ]:
train_df = train_df.drop(columns=['시간','연','월','일','도시','구','도시_x','도시_y'])
test_df = test_df.drop(columns=['시간','연','월','일','도시','구','도시_x','도시_y','ID','시군구'])

In [ ]:
categorical_features = ['요일','도로형태1','도로형태2','사고유형','기상상태', '노면상태','season','사고 발생 시간대']

for i in categorical_features:
    tr_encoder = TargetEncoder(cols=[i])
    train_df[i] = tr_encoder.fit_transform(train_df[i], train_df['ECLO'])
    test_df[i] = tr_encoder.transform(test_df[i])

In [ ]:
train_df = train_df.drop(columns=['군_남구','군_달서구','군_달성군','군_동구','군_북구','군_서구','군_수성구','군_중구'])
test_df = test_df.drop(columns=['군_남구','군_달서구','군_달성군','군_동구','군_북구','군_서구','군_수성구','군_중구'])

In [ ]:
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

In [ ]:
y_train = train_df['ECLO']

# train, test 데이터간 컬럼 동기화
cols = test_df.columns
X_train = train_df[cols]
X_test = test_df

In [ ]:
!pip install lightautoml

In [ ]:
pip install --upgrade log-calls

In [ ]:
pip install --upgrade lightautoml

  Using cached LightAutoML-0.2.16-py3-none-any.whl (262 kB)
INFO: pip is looking at multiple versions of lightautoml to determine which version is compatible with other requirements. This could take a while.


In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred, **kwargs):
    return mean_squared_error(y_true, y_pred, squared = False, **kwargs)

task = Task('reg', )

TypeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib
matplotlib.use('Agg')

In [ ]:
automl = AutoML(mode="Compete",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,total_time_limit=43200, eval_metric="rmse", ml_task = "regression",)
#3.209155

In [ ]:
automl.fit(X_train, y_train)

AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
There was an error during 1_DecisionTree training.
Please check AutoML_2/errors.md for details.
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models
There was an error during 1_Default_LightGBM training.
Please check AutoML_2/errors.md for details.
There was an error during 2_Default_Xgboost training.
Please check AutoML_2/errors.md for details.
There was an error during 3

ERROR:supervised.exceptions:No models produced. 
Please check your data or submit a Github issue at https://github.com/mljar/mljar-supervised/issues/new.


There was an error during 26_CatBoost training.
Please check AutoML_2/errors.md for details.
There was an error during 35_RandomForest training.
Please check AutoML_2/errors.md for details.
There was an error during 18_LightGBM training.
Please check AutoML_2/errors.md for details.
There was an error during 9_Xgboost training.
Please check AutoML_2/errors.md for details.
There was an error during 27_CatBoost training.
Please check AutoML_2/errors.md for details.
There was an error during 36_RandomForest training.
Please check AutoML_2/errors.md for details.
Skip golden_features because no parameters were generated.
'score' Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/supervised/tuner/mljar_tuner.py", line 225, in generate_params
    return self.get_kmeans_features_params(
  File "/usr/local/lib/python3.10/dist-packages/supervised/tuner/mljar_tuner.py", line 1188, in get_kmeans_features_params
    df_models, algorithms = self.df_models_algorithms(
 

AutoMLException: ignored

In [ ]:
pred = automl.predict(X_test)

NameError: ignored

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/sample_submission.csv')

submission['ECLO'] = pred
submission.loc[ submission['ECLO'] < 0.0, 'ECLO'] = 0.0
min(submission['ECLO'])

2.69994854927063

In [ ]:
submission.to_csv('/content/drive/MyDrive/대구 교통사고 피해 예측 경진대회/automl_ss3.csv', index=False)